In [2]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [3]:
import pickle

In [ ]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

In [5]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [8]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration'] = df.duration.dt.total_seconds() / 60

In [12]:
df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

In [13]:
categorical = ['PULocationID', 'DOLocationID']

In [14]:
df[categorical] = df[categorical].astype(str)

In [15]:
train_dicts = df[categorical].to_dict(orient='records')

In [16]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [18]:
target = 'duration'
y_train = df[target].values

In [21]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].astype('str')
    
    return df

In [22]:
df_val = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [23]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [24]:
X_val = dv.transform(val_dicts) 
y_val = df_val.duration.values

In [27]:
with open('mlruns/models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)


In [29]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [30]:
from hyperopt.pyll import scope

In [ ]:
train = xgb.DMatrix(X_train, label = y_train)
valid = xgb.DMatrix(X_val, label = y_val)

: 

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

[0]	validation-rmse:7.71910                           
[1]	validation-rmse:6.73898                           
[2]	validation-rmse:6.52086                           
[3]	validation-rmse:6.31215                           
[4]	validation-rmse:6.08884                           
[5]	validation-rmse:5.95123                           
[6]	validation-rmse:5.86637                           
[7]	validation-rmse:5.65369                           
[8]	validation-rmse:5.63260                           
[9]	validation-rmse:5.54945                           
[10]	validation-rmse:5.51550                          
[11]	validation-rmse:5.49497                          
[12]	validation-rmse:5.48444                          
[13]	validation-rmse:5.44131                          
[14]	validation-rmse:5.36762                          
[15]	validation-rmse:5.36106                          
[16]	validation-rmse:5.35189                          
[17]	validation-rmse:5.32392                          
[18]	valid

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)